In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport beanmachine.ppl

In [142]:
%autoreload

In [143]:
import beanmachine.ppl as bm
import torch
import torch.distributions as dist
import torch.optim as optim

class NormalNormal:
    def __init__(self):
        self.device = 'cpu'

    @bm.random_variable
    def mu(self):
        return dist.Normal(
            torch.zeros(1).to(self.device), 10 * torch.ones(1).to(self.device)
        )

    @bm.random_variable
    def x(self, i):
        return dist.Normal(self.mu(), torch.ones(1).to(self.device))

model = NormalNormal()

@bm.param
def phi():
    return torch.zeros(2)  # mean, log std

@bm.random_variable
def q_mu():
    params = phi()
    print(params)
    return dist.Normal(params[0], params[1].exp())

world = bm.VariationalInfer().infer(
    queries_to_guides={model.mu(): q_mu()},
    observations={
        model.x(1): torch.tensor(9.0),
        model.x(2): torch.tensor(10.0),
    },
    num_steps=100,
    optimizer=lambda params: optim.Adam(params, lr=1e-1),
)

  0%|          | 0/100 [00:00<?, ?it/s]

tensor(85.0159)
{RVIdentifier(wrapper=<function phi at 0x7f90262811f0>, arguments=()): tensor([0., 0.], requires_grad=True)}


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [116]:
world._run_node(q_mu())[0]

Normal(loc: -1.4006761312484741, scale: 25045.5)